# First part of the process : the query

## Initializaton of the Selenium webdriver

We will use Selenium with Firefox. We install before to start selenium with 
 > pip install selenium
 
We do install too Pandas that we will need at the end of the script :
> pip install pandas


In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
import pandas as pd

import time


# Set up selenium driver
driver = webdriver.Firefox()
url = "https://www.kayak.com/flights"
driver.get(url)
time.sleep(1)

# Close cookies pop-up
accept_cookies_xpath = '/html/body/div[12]/div/div[3]/div/div/div/div/div[1]/div/div[2]/div[2]/div[1]/button/span'
try:
    driver.find_element_by_xpath(accept_cookies_xpath).click()
except NoSuchElementException:
    pass

## Set the flights search

In [2]:
#########################################################
#  1st part of the process the query
#########################################################

# Settings the vars for que query
departure = 'Paris'
arrival = 'Tokyo'
departure_date = '07/09/2021'
returning_date = '07/20/2021'

# Origin
from_click_xpath = '//*[contains(@id, "origin-airport-display")]'
from_text_xpath = '//input[contains(@id, "origin-airport")]'
driver.find_element_by_xpath(from_click_xpath).click()
driver.find_element_by_xpath(from_text_xpath).send_keys(Keys.BACKSPACE + Keys.BACKSPACE + departure)
time.sleep(0.5)
driver.find_element_by_xpath(from_text_xpath).send_keys(Keys.RETURN)
time.sleep(2)

# To
to_click_xpath = '//*[contains(@id, "destination-airport-display")]'
to_text_xpath = r'//input[contains(@id, "destination-airport")]'
driver.find_element_by_xpath(to_click_xpath).click()
driver.find_element_by_xpath(to_text_xpath).send_keys(arrival)
time.sleep(1)
driver.find_element_by_xpath(to_text_xpath).send_keys(Keys.RETURN)
time.sleep(1)

# From Date
date_picker_from_xpath = '//*[contains(@id, "dateRangeInput-display-start-inner")]'
from_date_click_xpath = '//div[contains(@id, "depart")]'
from_date_text_xpath = '//div[contains(@id, "depart-input")]'
from_flexible_xpath = '//select[contains(@id, "datePicker-plusMinusThreeDepart-select")]'
driver.find_element_by_xpath(date_picker_from_xpath).click()
time.sleep(1)
driver.find_element_by_xpath(from_date_text_xpath).clear()
driver.find_element_by_xpath(from_date_text_xpath).send_keys(departure_date)

# Add flexible date option +/- 3days
#Select(driver.find_element_by_xpath(from_flexible_xpath)).select_by_value('plusminusthree')
time.sleep(1)

# To Date
to_date_text_xpath = '//div[contains(@id, "return-input")]'
to_flexible_xpath = '//select[contains(@id, "datePicker-plusMinusThreeReturn-select")]'
driver.find_element_by_xpath(to_date_text_xpath).clear()
time.sleep(1)
driver.find_element_by_xpath(to_date_text_xpath).send_keys(returning_date)
time.sleep(1)

# Add flexible date option +/- 3days
Select(driver.find_element_by_xpath(to_flexible_xpath)).select_by_value('plusminusthree')

# Click the submit button to launch the search
submit_button_xpath = '//button[contains(@id, "submit")]'
driver.find_element_by_xpath(submit_button_xpath).click()


## Wait the end of the research :

In [3]:
# Wait for the query result

class ProgressbarIsFull(object):
    """ An expectation for checking that the progressbar is present on the DOM of a page and that it is fully loaded
  """

    def __init__(self, locator):
        self.locator = locator

    def __call__(self, driver):
        progressbar = driver.find_element(*self.locator)
        return '(100%)' in progressbar.get_attribute('style')


progressbar_xpath = '//div[contains(@id, "-progress-bar")]/*/div[contains(@id, "bar")]'
driver_wait = WebDriverWait(driver, 30)
driver_wait.until(ProgressbarIsFull((By.XPATH, progressbar_xpath)))

True

In [4]:
## Close price alert 
close_alert_xpath = '//*[contains(@id, "dialog-content")]/*[contains(@id, "dialog-close")]'
for el in driver.find_elements_by_xpath(close_alert_xpath):
    if el.is_displayed():
        el.click()

# Second part of the process filter on the results :

In [5]:
#########################################################
# 2 nd part of the process filters on the result
#########################################################

# functions for filter with invisible element
def activate(driver, input_xpath, clickable_xpath):
    input = driver.find_element_by_xpath(input_xpath)
    if not input.get_attribute('checked'):
        div = driver.find_element_by_xpath(clickable_xpath)
        driver.execute_script("arguments[0].click();", div)


def deactivate(driver, input_xpath, clickable_xpath):
    input = driver.find_element_by_xpath(input_xpath)
    if input.get_attribute('checked'):
        div = driver.find_element_by_xpath(clickable_xpath)
        driver.execute_script("arguments[0].click();", div)

We don't want flights with 2 stops or more so we uncheck this settings on the left column

In [6]:
two_stop_xpath = '//input[contains(@id, "stops") and contains(@id, "2-check")]'
two_stop_click_xpath = '//div[contains(@id, "stops") and contains(@id, "2-check")]'
deactivate(driver, two_stop_xpath, two_stop_click_xpath)

# Third part get the data into a dataframe

In [7]:
############################################################
#  3 rd part of the process put the data into dataframe
############################################################


###################################
# Get Dates
xp_dates = '//div[@class="section date"]'
dates = driver.find_elements_by_xpath(xp_dates)
dates_list = [value.text for value in dates]

section_outbound_list = dates_list[::2]  # This is to separate the two flights outbound
section_return_list = dates_list[1::2]  # This is to separate the two flights return

# Inbound day and weekday
try:
    outbound_day = [value.split()[0] for value in section_outbound_list]
except IndexError:
    outbound_day = section_outbound_list[0].split()[0]

try:
    outbound_weekday = [value.split()[1] for value in section_outbound_list]
except IndexError:
    outbound_weekday = section_outbound_list[0].split()[1]

print(outbound_day)
print(outbound_weekday)

# Return day and weekday
try:
    return_day = [value.split()[0] for value in section_return_list]
except IndexError:
    try:
        return_day = section_return_list[0].split()[0]
    except IndexError:
        return_day = outbound_day


try:
    return_weekday = [value.split()[1] for value in section_return_list]
except IndexError:
    try:
        return_weekday = section_return_list[0].split()[1]
    except IndexError:
        return_weekday = outbound_weekday

print(return_day)
print(return_weekday)

['7/9', '7/9', '7/9', '7/9', '7/9', '7/9', '7/9', '7/9', '7/9', '7/9', '7/9', '7/9', '7/9', '7/9', '7/9', '7/9', '7/9']
['Fri', 'Fri', 'Fri', 'Fri', 'Fri', 'Fri', 'Fri', 'Fri', 'Fri', 'Fri', 'Fri', 'Fri', 'Fri', 'Fri', 'Fri', 'Fri', 'Fri']
['7/20', '7/20', '7/19', '7/21', '7/20', '7/21', '7/18', '7/20', '7/22', '7/19', '7/23', '7/17', '7/21', '7/18', '7/19', '7/20', '7/17']
['Tue', 'Tue', 'Mon', 'Wed', 'Tue', 'Wed', 'Sun', 'Tue', 'Thu', 'Mon', 'Fri', 'Sat', 'Wed', 'Sun', 'Mon', 'Tue', 'Sat']


In [8]:
###################################
# Get Airline company and departure and arrival times
xp_schedule = '//div[@class="section times"]'
schedules = driver.find_elements_by_xpath(xp_schedule)
hours_list = []
carrier_list = []
for schedule in schedules:
    hours_list.append(schedule.text.split('\n')[0])
    carrier_list.append(schedule.text.split('\n')[1])
# split the hours and carriers, between a and b legs
outbound_hours = hours_list[::2]
outbound_carrier = carrier_list[::2]
return_hours = hours_list[1::2]
return_carrier = carrier_list[1::2]

print(outbound_hours)
print(outbound_carrier)
print(return_hours)
print(return_carrier)

['10:45 am – 8:50 am +1', '10:45 am – 8:50 am +1', '7:00 pm – 1:55 pm +1', '7:00 pm – 1:55 pm +1', '7:00 pm – 1:55 pm +1', '1:25 pm – 8:30 am +1', '1:25 pm – 8:30 am +1', '1:25 pm – 8:30 am +1', '1:25 pm – 8:30 am +1', '1:25 pm – 8:30 am +1', '1:25 pm – 8:30 am +1', '1:25 pm – 8:30 am +1', '11:25 pm – 6:35 pm +1', '11:25 pm – 6:35 pm +1', '11:25 pm – 6:35 pm +1', '11:25 pm – 6:35 pm +1', '11:25 pm – 6:35 pm +1']
['LOT', 'LOT', 'Japan Airlines', 'Japan Airlines', 'Japan Airlines', 'Air France', 'Air France', 'Air France', 'Air France', 'Air France', 'Air France', 'Air France', 'Air France', 'Air France', 'Air France', 'Air France', 'Air France']
['10:15 am – 9:55 am +1', '10:15 am – 10:45 pm', '10:40 am – 4:15 pm', '10:40 am – 4:15 pm', '10:40 am – 4:15 pm', '10:35 am – 4:05 pm', '10:35 am – 4:05 pm', '10:35 am – 4:05 pm', '10:35 am – 4:05 pm', '10:35 am – 4:05 pm', '10:35 am – 4:05 pm', '10:35 am – 4:05 pm', '10:35 am – 4:05 pm', '10:35 am – 4:05 pm', '10:35 am – 4:05 pm', '10:35 am – 

In [9]:
###################################
# Get Flight duration and Airport IATA code
xp_airports = '//div[@class="section duration allow-multi-modal-icons"]'
airports = driver.find_elements_by_xpath(xp_airports)
airports_list = [value.text for value in airports]
airport_outbound_list = airports_list[::2]  # This is to separate the two flights
airport_return_list = airports_list[1::2]  # This is to separate the two flights

outbound_duration1 = []
outbound_airportDA = []
for n in airport_outbound_list:
    outbound_duration1.append(n[0:6])
    outbound_airportDA.append(''.join(n.split()[2:5]))

outbound_duration = [x.replace(' ', '') for x in outbound_duration1]

outbound_airportD = []  # Outbound Airport Departure
outbound_airportA = []  # Outbound Airport Arrival
for n in outbound_airportDA:
    outbound_airportD.append(n[0:3])
    outbound_airportA.append(n[4:7])

return_duration1 = []
return_airportDA = []
for n in airport_return_list:
    return_duration1.append(n[0:6])
    return_airportDA.append(''.join(n.split()[2:5]))

return_duration = [x.replace(' ', '') for x in return_duration1]

return_airportD = []
return_airportA = []
for n in return_airportDA:
    return_airportD.append(n[0:3])
    return_airportA.append(n[4:7])

print(outbound_duration)
print(return_duration)


['15h05', '15h05', '11h55', '11h55', '11h55', '12h05', '12h05', '12h05', '12h05', '12h05', '12h05', '12h05', '12h10', '12h10', '12h10', '12h10', '12h10']
['30h40', '19h30', '12h35', '12h35', '12h35', '12h30', '12h30', '12h30', '12h30', '12h30', '12h30', '12h30', '12h30', '12h30', '12h30', '12h30', '12h30']


In [10]:
###################################
# Prices
xp_prices = '//span[@class="price option-text"]/span[@class="price-text"]'
prices = driver.find_elements_by_xpath(xp_prices)
prices_list1 = [price.text for price in prices if price.text != '']
prices_list2 = [price[1:] for price in prices_list1]  # Remove the $
prices_list = [x.replace(',', '') for x in prices_list2]  # Remove the , separator
prices_list = [int(x) for x in prices_list]


print(prices_list)

[663, 617, 954, 954, 954, 997, 997, 997, 997, 997, 997, 997, 997, 997, 997, 997, 997]


In [13]:

cols = (
    ['Out Day', 'Out Weekday', 'Out Time', 'Out Airline', 'Out Airport D', 'Out Airport A', 'Out Duration',
     'Return Day',
     'Return Weekday',
     'Return Time', 'Return Airline', 'Return Airport D', 'Return Airport A', 'Return Duration',
     'Price'])

df = pd.DataFrame({'Out Day': outbound_day,
                   'Out Weekday': outbound_weekday,
                   'Out Time': outbound_hours,
                   'Out Airline': outbound_carrier,
                   'Out Airport D': outbound_airportD,
                   'Out Airport A': outbound_airportA,
                   'Out Duration': outbound_duration,
                   'Return Day': return_day,
                   'Return Weekday': return_weekday,
                   'Return Time': return_hours,
                   'Return Airline': return_carrier,
                   'Return Airport D': return_airportD,
                   'Return Airport A': return_airportA,
                   'Return Duration': return_duration,
                   'Price': prices_list})[cols]

# saving the dataframe
#df.to_excel('dataFlights.xlsx')

In [12]:
from IPython.display import display
display(df)

,Out Day,Out Weekday,Out Time,Out Airline,Out Airport D,Out Airport A,Out Duration,Return Day,Return Weekday,Return Time,Return Airline,Return Airport D,Return Airport A,Return Duration,Price
0,7/9,Fri,10:45 am – 8:50 am +1,LOT,CDG,NRT,15h05,7/20,Tue,10:15 am – 9:55 am +1,LOT,NRT,CDG,30h40,663
1,7/9,Fri,10:45 am – 8:50 am +1,LOT,CDG,NRT,15h05,7/20,Tue,10:15 am – 10:45 pm,LOT,NRT,CDG,19h30,617
2,7/9,Fri,7:00 pm – 1:55 pm +1,Japan Airlines,CDG,HND,11h55,7/19,Mon,10:40 am – 4:15 pm,Japan Airlines,HND,CDG,12h35,954
3,7/9,Fri,7:00 pm – 1:55 pm +1,Japan Airlines,CDG,HND,11h55,7/21,Wed,10:40 am – 4:15 pm,Japan Airlines,HND,CDG,12h35,954
4,7/9,Fri,7:00 pm – 1:55 pm +1,Japan Airlines,CDG,HND,11h55,7/20,Tue,10:40 am – 4:15 pm,Japan Airlines,HND,CDG,12h35,954
5,7/9,Fri,1:25 pm – 8:30 am +1,Air France,CDG,NRT,12h05,7/21,Wed,10:35 am – 4:05 pm,Air France,NRT,CDG,12h30,997
6,7/9,Fri,1:25 pm – 8:30 am +1,Air France,CDG,NRT,12h05,7/18,Sun,10:35 am – 4:05 pm,Air France,NRT,CDG,12h30,997
7,7/9,Fri,1:25 pm – 8:30 am +1,Air France,CDG,NRT,12h05,7/20,Tue,10:35 am – 4:05 pm,Air France,NRT,CDG,12h30,997
8,7/9,Fri,1:25 pm – 8:30 am +1,Air France,CDG,NRT,12h05,7/22,Thu,10:35 am – 4:05 pm,Air France,NRT,CDG,12h30,997
9,7/9,Fri,1:25 pm – 8:30 am +1,Air France,CDG,NRT,12h05,7/19,Mon,10:35 am – 4:05 pm,Air France,NRT,CDG,12h30,997
